In [74]:
import pandas as pd
import glob, os

In [75]:
df = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/flights_2021_all.csv')
df_w = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/ATL_2021.csv',parse_dates=['date'])

# df = pd.read_csv('../datasets/flights_2021_all.csv')
# df_w = pd.read_csv('../datasets/weather_data/ATL_2021.csv',parse_dates=['date'])

In [76]:
df.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15
0,0,2021,1,1,5,2021-01-05,11298,DFW,11433,DTW,1625,1623.0,0.0,0.0,2005,2000.0,0.0,0.0
1,1,2021,1,1,6,2021-01-06,11298,DFW,11433,DTW,1625,1628.0,3.0,0.0,2005,2005.0,0.0,0.0
2,2,2021,1,1,12,2021-01-12,11298,DFW,11433,DTW,1625,1620.0,0.0,0.0,2005,2000.0,0.0,0.0
3,3,2021,1,1,13,2021-01-13,11298,DFW,11433,DTW,1625,1621.0,0.0,0.0,2005,1954.0,0.0,0.0
4,4,2021,1,1,19,2021-01-19,11298,DFW,11433,DTW,1625,1625.0,0.0,0.0,2005,1944.0,0.0,0.0


# Datetime Conversion

- A new column <code>CombinedDateTime</code> by combining <code>FlightDate</code> column and   <code>CRSDepTime</code> column.
- Scale the column <code>ScaledCRSDepTime</code> to the nearest hour.

In [77]:
df['CRSDepTime'] = df['CRSDepTime'].astype('str')

In [78]:
df['CombinedDateTime'] = pd.to_datetime(df['FlightDate'] + ' ' + df['CRSDepTime'].str.zfill(4), format='%Y-%m-%d %H%M')

In [79]:
df['ScaledCRSDepTime'] = pd.to_datetime(df['CombinedDateTime']).dt.round('H')


In [80]:
df.sample(10)

,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,CombinedDateTime,ScaledCRSDepTime
229683,229683,2021,2,6,29,2021-06-29,14107,PHX,12892,LAX,1605,1600.0,0.0,0.0,1731,1722.0,0.0,0.0,2021-06-29 16:05:00,2021-06-29 16:00:00
95632,95632,2021,1,3,19,2021-03-19,11298,DFW,12266,IAH,2055,2050.0,0.0,0.0,2213,2150.0,0.0,0.0,2021-03-19 20:55:00,2021-03-19 21:00:00
267675,267675,2021,3,7,2,2021-07-02,11433,DTW,14771,SFO,857,903.0,6.0,0.0,1100,1050.0,0.0,0.0,2021-07-02 08:57:00,2021-07-02 09:00:00
45245,45245,2021,1,2,9,2021-02-09,11292,DEN,11298,DFW,1325,1321.0,0.0,0.0,1630,1610.0,0.0,0.0,2021-02-09 13:25:00,2021-02-09 13:00:00
143937,143937,2021,2,4,8,2021-04-08,11298,DFW,14107,PHX,905,904.0,0.0,0.0,954,947.0,0.0,0.0,2021-04-08 09:05:00,2021-04-08 09:00:00
383454,383454,2021,3,9,20,2021-09-20,14869,SLC,12892,LAX,1235,1254.0,19.0,1.0,1319,1336.0,17.0,1.0,2021-09-20 12:35:00,2021-09-20 13:00:00
350547,350547,2021,3,9,2,2021-09-02,12892,LAX,14100,PHL,1530,1521.0,0.0,0.0,2339,2322.0,0.0,0.0,2021-09-02 15:30:00,2021-09-02 16:00:00
215031,215031,2021,2,6,13,2021-06-13,11057,CLT,14107,PHX,1655,1702.0,7.0,0.0,1814,1813.0,0.0,0.0,2021-06-13 16:55:00,2021-06-13 17:00:00
164975,164975,2021,2,5,16,2021-05-16,14107,PHX,11292,DEN,917,913.0,0.0,0.0,1209,1200.0,0.0,0.0,2021-05-16 09:17:00,2021-05-16 09:00:00
51296,51296,2021,1,2,18,2021-02-18,11292,DEN,14747,SEA,1830,1818.0,0.0,0.0,2027,2033.0,6.0,0.0,2021-02-18 18:30:00,2021-02-18 18:00:00


# Weather data

In [81]:
csv_folder = '/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/'
for file_name in os.listdir(csv_folder):
    print(file_name)
    if file_name!='test':
        dff = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/'+file_name)
        dff.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)
        dff['Origin'] = file_name[:3]
        dff.to_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/test/'+file_name)

SEA_2021.csv
SLC_2021.csv
SFO_2021.csv
ORD_2021.csv
test
ATL_2021.csv
CLT_2021.csv
DEN_2021.csv
LAX_2021.csv
PHX_2021.csv
DTW_2021.csv
PHL_2021.csv
BWI_2021.csv
IAH_2021.csv
MDW_2021.csv
DFW_2021.csv


In [82]:
csv_folder = '/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/test/'
concatenated_data = pd.DataFrame()
for file_name in os.listdir(csv_folder):
    print(file_name)
    if file_name!='.DS_Store':
        df_ = pd.read_csv(csv_folder+file_name)
        concatenated_data = pd.concat([concatenated_data, df_], ignore_index=True)
concatenated_data.to_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_merged.csv')


SEA_2021.csv
.DS_Store
SLC_2021.csv
SFO_2021.csv
ORD_2021.csv
ATL_2021.csv
CLT_2021.csv
DEN_2021.csv
LAX_2021.csv
PHX_2021.csv
DTW_2021.csv
PHL_2021.csv
BWI_2021.csv
IAH_2021.csv
MDW_2021.csv
DFW_2021.csv


In [83]:
concatenated_data.columns

Index(['Unnamed: 0', 'ScaledCRSDepTime', 'temperature_2m',
       'relative_humidity_2m', 'dew_point_2m', 'precipitation', 'rain',
       'snowfall', 'weather_code', 'surface_pressure', 'cloud_cover',
       'wind_speed_10m', 'wind_direction_10m', 'Origin'],
      dtype='object')

In [84]:
df_w = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_merged.csv',parse_dates=['ScaledCRSDepTime'])


In [85]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131400 entries, 0 to 131399
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Unnamed: 0.1          131400 non-null  int64         
 1   Unnamed: 0            131400 non-null  int64         
 2   ScaledCRSDepTime      131400 non-null  datetime64[ns]
 3   temperature_2m        131400 non-null  float64       
 4   relative_humidity_2m  131400 non-null  float64       
 5   dew_point_2m          131400 non-null  float64       
 6   precipitation         131400 non-null  float64       
 7   rain                  131400 non-null  float64       
 8   snowfall              131400 non-null  float64       
 9   weather_code          131400 non-null  float64       
 10  surface_pressure      131400 non-null  float64       
 11  cloud_cover           131400 non-null  float64       
 12  wind_speed_10m        131400 non-null  float64       
 13 

In [86]:
df_w.head()

,Unnamed: 0.1,Unnamed: 0,ScaledCRSDepTime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m,Origin
0,0,0,2021-01-01 00:00:00,7.7745,92.752815,6.6745,0.0,0.0,0.0,3.0,1005.58594,100.0,15.273505,188.13002,SEA
1,1,1,2021-01-01 01:00:00,7.7245,91.798290,6.4745,0.0,0.0,0.0,3.0,1005.68200,100.0,12.371645,188.36580,SEA
2,2,2,2021-01-01 02:00:00,7.3245,92.408510,6.1745,0.0,0.0,0.0,3.0,1005.85860,100.0,11.212135,174.47255,SEA
3,3,3,2021-01-01 03:00:00,7.4245,90.208900,5.9245,0.0,0.0,0.0,3.0,1005.56824,100.0,11.966953,173.08887,SEA
4,4,4,2021-01-01 04:00:00,7.5245,88.977980,5.8245,0.0,0.0,0.0,3.0,1005.57330,100.0,12.245294,178.31535,SEA


In [87]:
df_w.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)

In [88]:
df_w.head()

,Unnamed: 0.1,Unnamed: 0,ScaledCRSDepTime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m,Origin
0,0,0,2021-01-01 00:00:00,7.7745,92.752815,6.6745,0.0,0.0,0.0,3.0,1005.58594,100.0,15.273505,188.13002,SEA
1,1,1,2021-01-01 01:00:00,7.7245,91.798290,6.4745,0.0,0.0,0.0,3.0,1005.68200,100.0,12.371645,188.36580,SEA
2,2,2,2021-01-01 02:00:00,7.3245,92.408510,6.1745,0.0,0.0,0.0,3.0,1005.85860,100.0,11.212135,174.47255,SEA
3,3,3,2021-01-01 03:00:00,7.4245,90.208900,5.9245,0.0,0.0,0.0,3.0,1005.56824,100.0,11.966953,173.08887,SEA
4,4,4,2021-01-01 04:00:00,7.5245,88.977980,5.8245,0.0,0.0,0.0,3.0,1005.57330,100.0,12.245294,178.31535,SEA


# Combine two datasets

In [89]:
df.columns 

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'ArrDel15', 'CombinedDateTime', 'ScaledCRSDepTime'],
      dtype='object')

In [90]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131400 entries, 0 to 131399
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Unnamed: 0.1          131400 non-null  int64         
 1   Unnamed: 0            131400 non-null  int64         
 2   ScaledCRSDepTime      131400 non-null  datetime64[ns]
 3   temperature_2m        131400 non-null  float64       
 4   relative_humidity_2m  131400 non-null  float64       
 5   dew_point_2m          131400 non-null  float64       
 6   precipitation         131400 non-null  float64       
 7   rain                  131400 non-null  float64       
 8   snowfall              131400 non-null  float64       
 9   weather_code          131400 non-null  float64       
 10  surface_pressure      131400 non-null  float64       
 11  cloud_cover           131400 non-null  float64       
 12  wind_speed_10m        131400 non-null  float64       
 13 

In [91]:
merged_df = pd.merge(df, df_w, on=['Origin', 'ScaledCRSDepTime'], suffixes=('', '_df2'))


In [92]:
# merge = pd.merge(df,df_w,on=['Origin','ScaledCRSDepTime'],how='left')

In [98]:
merged_df.columns

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'ArrDel15', 'CombinedDateTime', 'ScaledCRSDepTime',
       'Unnamed: 0.1', 'Unnamed: 0_df2', 'temperature_2m',
       'relative_humidity_2m', 'dew_point_2m', 'precipitation', 'rain',
       'snowfall', 'weather_code', 'surface_pressure', 'cloud_cover',
       'wind_speed_10m', 'wind_direction_10m'],
      dtype='object')

In [100]:
remove_cols = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 0_df2']
merged_df.drop(remove_cols, axis=1, inplace=True)
merged_df.shape

(538040, 30)

In [101]:
merged_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'ArrDel15', 'CombinedDateTime', 'ScaledCRSDepTime',
       'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'snowfall', 'weather_code', 'surface_pressure',
       'cloud_cover', 'wind_speed_10m', 'wind_direction_10m'],
      dtype='object')

In [ ]:
merged_df.to_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/final_dataset.csv')